In [ ]:
# !wget http://mireu-server.iptime.org/api/public/dl/QLwEhRq3/DataSet/%EA%B0%9D%EC%B2%B4%EC%9D%B8%EC%8B%9D_%EC%97%B0%EA%B5%AC/13.%ED%95%9C%EA%B5%AD%EC%96%B4%EA%B8%80%EC%9E%90%EC%B2%B4/image_gen/new_image/gen_image.tar.gz
# !tar -xzvf gen_image.tar.gz
!pip3 install tqdm

In [1]:
from PIL import Image
import numpy as np
import os

layerList = ["가", "나", "다"]
# , "라", "마", "바", "아", "자", "차", "카", "타", "파", "하"

all_x_array = []
all_y_array = []

for dir in layerList:
    fileList = os.listdir(f"../../dataset_dir/gen_image/{dir}")[:6000]

    for file in fileList:
        y_array = dict.fromkeys(layerList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성

        imageData = Image.open(f"""../../dataset_dir/gen_image/{dir}/{file}""") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((32, 32)) # 이미지 다운스케일링

        imageData = np.array(imageData)
        all_x_array.append(imageData.tolist())

        y_array[dir] = 1 # 사용자가 지정한 str 값을 1로 변경
        y_array = list(y_array.values())
        all_y_array.append(y_array)


In [2]:
x_array = np.array(all_x_array)
y_array = np.array(all_y_array)

In [3]:
print(x_array.shape, y_array.shape)

(13498, 50, 50) (13498, 13)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array, y_array, test_size=0.2, shuffle=False, random_state=1004
)

In [5]:
X_train = X_train.astype("float32") / 255
Y_train = Y_train.astype("float32") / 255

X_test = X_test.astype("float32") / 255
Y_test = Y_test.astype("float32") / 255

In [6]:
image_w, image_h = X_train.shape[1], X_train.shape[2]

X_train = X_train.reshape(-1, image_w, image_h, 1)
X_test = X_test.reshape(-1, image_w, image_h, 1)

In [7]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(10798, 50, 50, 1) (2700, 50, 50, 1)
(10798, 13) (2700, 13)


In [8]:
import tensorflow as tf

# tf.debugging.set_log_device_placement(True)

gpus = tf.config.list_physical_devices('GPU')
if gpus: # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')

  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

In [12]:
import tensorflow as tf
import keras

model = keras.Sequential([
    keras.Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    keras.layers.Conv2D(32, (5, 5), activation='relu'),
    keras.layers.MaxPooling2D((3, 3)),
    keras.layers.Conv2D(64, (4, 4), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(13, activation="softmax"),
])


In [13]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 46, 46, 32)        832       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_25 (Conv2D)          (None, 12, 12, 64)        32832     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 2, 2, 128)       

In [14]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.01), metrics=["accuracy"])

history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

Epoch 1/20
85/85 [==============================] - 6s 61ms/step - loss: 343924.5625 - accuracy: 0.0825 - val_loss: 1140822.8750 - val_accuracy: 0.0000e+00
Epoch 2/20
85/85 [==============================] - 5s 61ms/step - loss: 30100072.0000 - accuracy: 0.0783 - val_loss: 25593456.0000 - val_accuracy: 0.0000e+00
Epoch 3/20
85/85 [==============================] - 5s 60ms/step - loss: 305055296.0000 - accuracy: 0.0729 - val_loss: 411524128.0000 - val_accuracy: 0.0000e+00
Epoch 4/20
43/85 [==============>...............] - ETA: 2s - loss: 860748480.0000 - accuracy: 0.0814

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"], label="loss") 
plt.plot(history.history["accuracy"], label="accuracy") 
plt.legend()
plt.show()